In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_
import re
#from collections import Counter
import uuid 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import sqlite3

sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

In [12]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
events = Table('events', metadata, autoload = True, autoload_with = engine)

In [3]:
def cleanAction(x):
    x = re.sub('^\d+', '', x).lstrip()
    x = re.sub('\d+$', '', x).rstrip()
    x = re.sub(' by$', '', x).rstrip()
    return x
def cleanPeriod(x):
    x = int(''.join(filter(str.isdigit, x)))
    return x
def cleanPlayer(x):
    x = x.lstrip().rstrip()
    return x
def getStarters(df):
    periodStart = pd.to_timedelta('00:00:00')
    periodEnd = pd.to_timedelta('00:20:00')
    lineups2 = df[df.action.isin(['enters the game','goes to the bench'])
                 ][['player','action','time','period','team']]
    #lineups2['time'] = lineups2['time'].apply(pd.to_datetime) + pd.DateOffset(lineups2['period'])
    linePV = pd.pivot_table(lineups2,index=['player','team'],columns='action',values='time',aggfunc=np.min).reset_index()
    linePV['enters the game'] = linePV['enters the game'].fillna(periodStart)
    linePV['goes to the bench'] = linePV['goes to the bench'].fillna(periodEnd)
    
    starters = linePV[
        (
            (linePV['goes to the bench'] < linePV['enters the game'])
        )
        |
        (
            (linePV['enters the game'] == '00:00:00')
        )
                     ][['team','player']]
    #return set(starters[starters['team']=='Home']['player']),set(starters[starters['team']=='Away']['player'])
    return list(starters[starters['team']=='Home']['player']),list(starters[starters['team']=='Away']['player'])

def groupToList(df,k):
         keys, values = df.sort_values(k).values.T
         ukeys, index = np.unique(keys, True)
         arrays = np.split(values, index[1:])
         df2 = pd.DataFrame({k:ukeys, 'b':[list(a) for a in arrays]})
         return df2

In [4]:
actValMap = {
'Assist':0,
'Block':0,
'Foul':0,
'Steal':0,
'Technicaloul':0,
'Turnover':0,
'deadball rebound':0,
'defensive rebound':0,
'enters the game':0,
'goes to the bench':0,
'made 2-pt field goal':2,
'made 3-pt jump shot':3,
'made dunk':2,
'made free throw':1,
'made jump shot':2,
'made layup':2,
'made tip-in':2,
'missed 2-pt field goal':0,
'missed 3-pt jump shot':0,
'missed dunk':0,
'missed free throw':0,
'missed jump shot':0,
#missed jump shot3missed 3-pt jump shot2missed 2-pt field goalmissed layupmissed free throwmissed dunkmissed tip-in}	123
'missed layup':0,
'missed tip-in':0,
'offensive rebound':0,
'pointswrong basket by defense':-2,
#score=2text=made jump shot3score=3text=made 3-pt jump shot2score=2text=made 2-pt field goalscore=2text=made layupscore=1text=made free throwscore=2text=made dunkscore=2text=made tip-in	108
'will be starting':0
}

In [7]:
def fetchGameMeta():
    stmt3 = "SELECT * FROM games WHERE isProcessed=1 AND (pmProcessed = 0 or pmProcessed IS NULL) LIMIT 1 "
    res3 = conn.execute(stmt3).fetchall()
    gameID = res3[0][0]
    homeTeam = res3[0][1]
    homeScore_final = res3[0][2]
    awayTeam = res3[0][3]
    awayScore_final = res3[0][4]
    gameDate = res3[0][5]
    return gameID,homeTeam,homeScore_final,awayTeam,awayScore_final,gameDate

def fetchGame(gameID):    
    stmt2 = "SELECT * FROM events WHERE gameid = '" + gameID + "'"
    df = pd.read_sql_query(stmt2,engine)
    return df

def cleanGame(df):
    df['scoreHome'] = df['scoreHome'].map(int)
    df['scoreAway'] = df['scoreAway'].map(int)
    
    df['action'] = df['action'].apply(cleanAction)
    df['period'] = df['period'].apply(cleanPeriod).apply(int)
    df['player'] = df['player'].apply(cleanPlayer)
    df['duration'] = df['duration'].apply(int)
    
    df['actionValue'] = df['action'].map(actValMap).map(int,na_action='ignore')
    
    df['time'] = pd.to_timedelta('00:'+df['time'])
    df.loc[df['period'] <= 2,'time'] = pd.to_timedelta('00:20:00') - df.loc[df['period'] <= 2,'time']
    df.loc[df['period'] > 2,'time'] = pd.to_timedelta('00:05:00') - df.loc[df['period'] > 2,'time']
    df.loc[df['period'] == 2,'time'] += pd.to_timedelta('00:20:00')
    df.loc[df['period'] == 3,'time'] += pd.to_timedelta('00:25:00')
    df.loc[df['period'] == 4,'time'] += pd.to_timedelta('00:30:00')
    df.loc[df['period'] == 5,'time'] += pd.to_timedelta('00:35:00')
    df.loc[df['period'] == 6,'time'] += pd.to_timedelta('00:40:00')
    df.loc[df['period'] == 7,'time'] += pd.to_timedelta('00:45:00')
    df.loc[df['period'] == 8,'time'] += pd.to_timedelta('00:50:00')
    
    df['seqNo'] = df['time'].ne(df['time'].shift()).cumsum()
    
    df = df.sort_values(by=['time'],ascending=True)
    
    conditions = [
        (df['actionValue'] == 1),
        (df['actionValue'] == 2),
        (df['actionValue'] == 3),
        (df['action'].str.contains('missed') & df['action'].str.contains('3')),
        (df['action'].str.contains('missed') & ~df['action'].str.contains('3') & ~df['action'].str.contains('free')),
        (df['action'].str.contains('missed') & ~df['action'].str.contains('3') & df['action'].str.contains('free'))
    ]
    choices = ['FTM', 'FG2', 'FG3','3PA','2PA','FTA']
    
    df['action_edit1'] = np.select(conditions, choices, default=df['action'])
    
    #df['psnChg'] = False
    
    df['playScore'] = df['time'].map(df.groupby("time")['actionValue'].sum())
    return df

def sequenceGame(df):
    metricDF1 = df.copy()
    metricDF1.loc[metricDF1['team']=='Away','actionValue'] = metricDF1.loc[metricDF1['team']=='Away','actionValue'] * -1
    metricDF1 = metricDF1.set_index('team').groupby('seqNo', as_index=False).agg({
        #'action':dict
        'actionValue':np.sum
        , 'duration':np.sum
        , 'scoreHome':np.max
        , 'scoreAway':np.max
    }
    )
    metricDF1['n_1'] = metricDF1['actionValue'].shift(-1)
    
    metricDF1['clockUse'] = pd.cut(metricDF1['duration'],5,labels = ['Early','Early-Mid','Mid','Late-Mid','Late'])
    return metricDF1

def set_pm(df,debug=False):
    HLU,ALU = getStarters(df)
    lineupDF = df.copy().loc[df.action.isin(['enters the game','goes to the bench']),['time','action','player','team','scoreHome','scoreAway','seqNo']]
    lineupDF = lineupDF.reset_index()
    
    rosterH = [lineupDF.player.unique.where(lineupDF.team=='Home')]
    rosterA = [lineupDF.player.unique.where(lineupDF.team=='Away')]
    
    seq = lineupDF.loc[0,'seqNo'].copy()
    time = lineupDF.loc[0,'time']
    hSc = lineupDF.loc[0,'scoreHome'].copy()
    aSc = lineupDF.loc[0,'scoreAway'].copy()
    diff = hSc-aSc
    
    away = pd.DataFrame(data={'lineup':[ALU],'time':pd.to_timedelta('00:00:00'),'team':'Away','diff':0}).head(1)
    home = pd.DataFrame(data={'lineup':[HLU],'time':pd.to_timedelta('00:00:00'),'team':'Home','diff':0}).head(1)
    
    h = home.loc[0,'lineup'].copy()
    a = away.loc[0,'lineup'].copy()
    
    hPlayerPM = {'Home':{i:{'curDiff':0, 'pm':0, 'curTime':pd.to_timedelta('00:00:00'), 'tmp':0} for i in lineupDF[lineupDF['team']=='Home'].player.unique()}}
    aPlayerPM = {'Away':{i:{'curDiff':0, 'pm':0, 'curTime':pd.to_timedelta('00:00:00'), 'tmp':0} for i in lineupDF[lineupDF['team']=='Away'].player.unique()}}
    aPlayerPM.update(hPlayerPM)
    playerPM = aPlayerPM
    if debug:
        print('Home On Floor: ' + str(h))
        print('Away On Floor: ' + str(a))
    
    stints = pd.DataFrame(data = {'seqStart':0
                                  , 'seqEnd':seq
                                  , 'timeStart':pd.to_timedelta('00:00:00')
                                  , 'timeEnd':time
                                  , 'diffStart':0
                                  , 'diffEnd': diff
                                  , 'HOF':[[p for p in h]]
                                  , 'AOF':[[p for p in a]]
                                  , 'HOB':[[p for p in rosterH if p not in h]]
                                  , 'HOA':[[p for p in rosterA if p not in a]]
                                  , 'HX':[[1 for p in rosterH if p in h else 0]]
                                  , 'AX':[[1- for p in rosterA if p in a else 0]]
                                  ,' Hdiff':[0 for i in rosterH]
                                  ,' Adiff':[0 for i in rosterA]
                                  , 'HR':[rosterH]
                                  , 'AR':[rosterA]
                                  , 'Y':((diffEnd - diffStart)/(seqEnd - seqStart))/100
                                 })
    
    
    hx = [1 for p in rosterH if p in h else 0]
    ax = [1 for p in rosterA if p in a else 0]
    
    for i in range(0,len(lineupDF)):
        nseq = lineupDF.loc[i,'seqNo'].copy()
        ntime = lineupDF.loc[i,'time']
        plyr = lineupDF.loc[i,'player']
        act = lineupDF.loc[i,'action']
        tm = lineupDF.loc[i,'team']
        ndiff = diff.copy()
        #print(seq)
        #print(nseq)
        if nseq != seq:
            hSc = lineupDF.loc[i,'scoreHome'].copy()
            aSc = lineupDF.loc[i,'scoreAway'].copy()
            ndiff = hSc - aSc
            if tm=="Away":
                ndiff *= -1
            if homeSub:
                try:
                    newH = pd.DataFrame(data={'lineup':[[p for p in h]],'time':time,'team':'Home','diff':hSc-aSc},index=[0])
                    home = home.append(newH)
                    if debug:
                        print('Added ' + str(newH.loc[0,'lineup']) + ' to Home lineups')
                except ValueError as e:
                    if debug:
                        print('Couldn''t add ' + str(newH))
                        print(e)
            if awaySub:
                try:
                    newA = pd.DataFrame(data={'lineup':[[p for p in a]],'time':time,'team':'Away','diff':aSc-hSc},index=[0])#.head(1)
                    away = away.append(newA)
                    if debug:
                        print('Added ' + str(newA.loc[0,'lineup']) + ' to Away lineups')
                except ValueError as e:
                    if debug:
                        print('Couldn''t add ' + str(newA))
                        print(e)
            hx_cur = []
            ax_cur = []
            hx_cur = [1 for p in rosterH if p in h else 0]
            ax_cur = [-1 for p in rosterA if p in a else 0]
            newStint = pd.DataFrame(data={'seqStart':seq
                                          , 'seqEnd':nseq
                                          , 'timeStart':time
                                          , 'timeEnd':ntime
                                          , 'diffStart':diff
                                          , 'diffEnd': ndiff
                                          , 'HOF': [[p for p in h]]
                                          , 'AOF': [[p for p in a]]
                                          , 'HOB':[[p for p in rosterH if p not in h]]
                                          , 'HOA':[[p for p in rosterA if p not in a]]
                                          , 'HX':[hx_cur]
                                          , 'AX':[ax_cur]
                                          , 'HR':[rosterH]
                                          , 'AR':[rosterA]
                                          , 'Y':((diffEnd - diffStart)/(seqEnd - seqStart))/100
                                                 }
                                           )
            stints = stints.append(newStint)
            seq = nseq
            time = ntime
            diff = ndiff
        if debug:
            print(ntime)
            print(tm + ':' + plyr + ' ' + act)
        homeSub = False
        awaySub = False
        if act=='enters the game':
            if tm=='Home' and plyr not in h:
                h.append(plyr)
                homeSub = True
                #print('Added ' + lineupDF.loc[i,'player'])
            else:
                if plyr not in h:
                    a.append(plyr)
                    awaySub = True
                    #print('Added ' + lineupDF.loc[i,'player'])
            playerPM[tm][plyr]['curTime']=ntime
            playerPM[tm][plyr]['curDiff']=ndiff
        if act=='goes to the bench':
            if tm=='Home':
                try:
                    h.remove(plyr)
                    homeSub = True
                    #print('Removed ' + lineupDF.loc[i,'player'])
                except Exception as e:
                    if debug:
                        print('Exception: ' + str(e))
            else:
                try:
                    a.remove(plyr)
                    awaySub = True
                    #print('Removed ' + lineupDF.loc[i,'player'])
                except Exception as e:
                    if debug:
                        print('Exception :' + str(e))
            playerPM[tm][plyr]['pm'] += (ndiff - playerPM[tm][plyr]['curDiff'])
            #print((ndiff - playerPM[tm][plyr]['curDiff']))
            timeEntered = playerPM[tm][plyr]['curTime']    
            minPlayed = ((ntime - timeEntered).seconds)/60
            
            playerPM[tm][plyr]['tmp'] += int(minPlayed)
            if debug:
                print('Time: ',(ntime))
                print('Player entered at: ',(playerPM[tm][plyr]['curTime']))
                print('Minutes Played: ',(ntime - playerPM[tm][plyr]['curTime']))
        h.sort()
        a.sort()
        if debug:
            print('Home On Floor: ' + str(h))
            print('Away On Floor: ' + str(a))
    return playerPM

def updatePlayers(playerPM,gameID,homeTeam,awayTeam):
    pmdfH = pd.DataFrame(data=playerPM.copy()['Home']).transpose().reset_index().iloc[:,[0,2,4]]
    pmdfH['team'] = homeTeam
    pmdfA = pd.DataFrame(data=playerPM.copy()['Away']).transpose().reset_index().iloc[:,[0,2,4]]
    pmdfA['team'] = awayTeam
    
    pmDF = pd.concat([pmdfH,pmdfA])
    pmDF['year'] = gameDate[0:4]
    pmDF.rename(columns={'index':'name'}, inplace=True)
    pmDF['id'] = pmDF['team'] + '_' + pmDF['name']
    pmDF = pmDF[['id','name','team','year','pm','tmp']]
    pmDF['tmp'] = pmDF['tmp'].apply(int)
    #"""
    pmDF.to_sql('temp_table', engine, if_exists='replace')
    
    sql = """
    INSERT INTO players
    SELECT
    id,name,team,year,pm,tmp
    FROM temp_table AS tt 
    WHERE true
      ON CONFLICT(id) DO UPDATE SET pm = pm + excluded.pm, tmp = tmp + excluded.tmp;
    """
    try:
        conn.execute(sql)
        stmt = games.update().where(games.c.id==gameID).values(pmProcessed=True)
        conn.execute(stmt)
        print('Processed game ID: ' + gameID)
    except Exception as e:
        print('Error writing game :' + gameID)

In [ ]:
stmt = select([games.c.id]).where((games.columns.isProcessed == True) 
                                  & (games.columns.pmProcessed == False)
                                  & (games.columns.pmFailed != True)
                                 )
results = conn.execute(stmt).fetchall()
gameIDs = [value for value, in results]


N = len(gameIDs)
n = 0
print(N)
if N > 1000:
    N = 1000

while n < N:
    gameID,homeTeam,homeScore_final,awayTeam,awayScore_final,gameDate = fetchGameMeta()
    try:        
        print('Processing ' + str(n) + '/' + str(N))
        df = fetchGame(gameID)
        df = cleanGame(df)
        seqdf = sequenceGame(df)
        playerPM = set_pm(df)
        updatePlayers(playerPM,gameID,homeTeam,awayTeam)
    except:
        print('Failed processing game ID: ' + gameID)
        stmt = games.update().where(games.c.id==gameID).values(pmFailed=True)
        conn.execute(stmt)
        stmt = games.update().where(games.c.id==gameID).values(pmProcessed=True)
        conn.execute(stmt)
    games = Table('games', metadata, autoload = True, autoload_with = engine)
    n += 1


In [ ]:
from sklearn.linear_model import Ridge

X = pd.DataFrame(data={'X':stints['HX'] + stints['AX']})
#X['X'] = X['X'].shift(1) - X['X']
Y = stints['Y'] 
#Y['Y'] = Y['Y'].shift(1) - Y['Y']
#Y = Y * X.T
#X = X * X.T

clf = Ridge(alpha=1.0)
cfit = clf.fit(X, y)
cfit.getParams()


In [ ]:
gameID = '20200104_xnf4'
df = fetchGame(gameID)
df = cleanGame(df)

#set_pm(df,True)
periodStart = pd.to_timedelta('00:00:00')
periodEnd = pd.to_timedelta('00:20:00')
lineups2 = df[df.action.isin(['enters the game','goes to the bench'])
             ][['player','action','time','period','team']]
#lineups2['time'] = lineups2['time'].apply(pd.to_datetime) + pd.DateOffset(lineups2['period'])
linePV = pd.pivot_table(lineups2,index=['player','team'],columns='action',values='time',aggfunc=np.min).reset_index()
"""
linePV['enters the game'] = linePV['enters the game'].fillna(periodStart)
linePV['goes to the bench'] = linePV['goes to the bench'].fillna(periodEnd)

starters = linePV[
    (
        (linePV['goes to the bench'] < linePV['enters the game'])
    )
    |
    (
        (linePV['enters the game'] == '00:00:00')
    )
                 ][['team','player']]
"""
df.action.unique()

#metricDF1.head(50)
#df.head(50)
#df.action.unique()

In [ ]:
HLU = []
ALU = []
HLU, ALU = getStarters(df)

endScore_calc = df.scoreHome.iat[-1]
endScore_calc = df.scoreAway.iat[-1]

df.head(10)

In [ ]:
rebDF = df[df.action.str.contains('rebound')].copy()
rebDF.merge(metricDF1.assign(seqNo=metricDF1.seqNo-1),on='seqNo')[[
    'id'
    , 'gameid'
    , 'time'
    , 'action_x'
#    , 'scoreHome_x'
#    , 'scoreAway_x'
    , 'team'
    , 'duration_x'
    , 'duration_y'
    , 'player'
    , 'period'
    , 'playScore'
#    , 'actionValue_x'
    , 'seqNo'
#    , 'action_edit1'
#    , 'psnChg'
    , 'action_y'
    , 'actionValue_y'
    , 'scoreHome_y'
    , 'scoreAway_y'
#    , 'n_1'
    , 'clockUse'
]]

#rebDF.head(25)

In [ ]:
#view all actions and frequency
#"""
stmt = "\
SELECT \
action\
,COUNT(1) \
FROM events \
WHERE action NOT LIKE '%time%out%'\
GROUP BY \
action \
ORDER BY 2 desc"

act = pd.read_sql_query(stmt,engine)
#"""

act['action'] = act['action'].apply(cleanAction)

act = act.groupby('action').sum()

act.head(50)#.action.unique()#.head(50)

In [ ]:
stmt = "SELECT DISTINCT \
e.gameid\
,e.period\
,e.action AS action1 \
,e2.action AS action2 \
,e.team AS team1 \
,e2.team AS team2 \
FROM events AS e \
INNER JOIN events AS e2 \
ON e.gameid = e2.gameid \
AND e.period = e2.period \
AND e.time = e2.time \
AND e.action > e2.action\
 WHERE e.gameid = '" + gameIDs[0] + "'" 


#print(stmt)
actdf = pd.read_sql_query(stmt,engine)
actdf['action1'] = actdf.action1.apply(cleanAction)
actdf['action2'] = actdf.action2.apply(cleanAction)

actdf.head(25)

In [ ]:
#stmt = "DELETE FROM events"
#conn.execute("DELETE FROM players")

In [ ]:
stmt = "SELECT * FROM games"
res = conn.execute(stmt).fetchall()
res[0:20]

In [ ]:
stmt = games.update().values(pmFailed=False)
conn.execute(stmt)

In [14]:
conn.close()

In [ ]:
stmt = select([games]).where((games.columns.pmFailed == True)
                                 )
results = conn.execute(stmt).fetchall()
results